### 1. Load Raw Data

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#load raw data
users = pd.read_csv('data/USERS.csv')
qs = pd.read_csv('data/QUESTIONNAIRE.csv')
motion = pd.read_csv('data/MOTION.csv')
sound = pd.read_csv('data/SOUND.csv')
location = pd.read_csv('data/LOCATION.csv')

#remove user ids under 30...these were test users
users = users[users['userID'] >= 30]
qs = qs[qs['user'] >= 30]
motion = motion[motion['user'] >= 30]
sound = sound[sound['user']>= 30]
location = location[location['user'] >= 30]

#convert datetime to pandas datetime format and remove any rows that could not be converted (dropping NAN times is extremely rare)
motion['datetime'] = pd.to_datetime(motion['datetime'],infer_datetime_format=True,errors='coerce')
motion = motion.dropna(subset=['datetime'])
#remove columns that are not useful
motion = motion.drop(['motionEntryID','f1','f34','f35'],axis=1)


#Remove Questionnaire entries that do not have a enough valid answers
#(Result1 and Result2 are the SF36 questionnaire summary values for Physical Component Score and Mental component scores.
#They are computed using individual questions Q1-Q37
#To ensure that Result1 and Result2 are valid, we first check that enough of the individual questions have been answered
#This is done by selecting just the individual questions (qs.iloc[:,3:39]) and count how many are not -1
#If enought are valid (we set 18 as an arbitrary threshold), then we keep that record
qs = qs[(qs.iloc[:,3:39] > 0).sum(axis=1) > 18]

#If a user has multiple questionnaire entries...keep only the most recent (tail) one
qs = qs.groupby(by='user').tail(1)

#Join Questionnaire with User (do a qs left join with users so that only users with valid questionnaire data are kept in the resulting join)
users = users.rename(columns={'userID':'user'})
valid_questionnaire = pd.merge(qs,users,on='user',how='left')

#Join valid users from questionnaire with motion dataframe...effectively removing users from the motion dataframe that do not have valid questionnaire data
motion = pd.merge(valid_questionnaire[['user']],motion,on='user',how='left')

### 2. Motion File Processing:

##### 2.1 Users that have uploaded less than 40 hours of movement data (40 hours when phone was not stationary) will be removed
##### f2-28 features will be NAN for hours where NO movement occured (details <a href="featureDescriptions.md">here</a>). We use f2 the count hours of non-NAN values

,datetime,qEntryID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,Q34,Q35,Q36,Result1,Result2,Age,Gender,deviceID,registerDate,countryID
user,,,,,,,,,,,,,,,,,,,,,
31,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
34,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
36,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
37,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
41,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
5241,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5253,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [17]:
#Count number of f2 features for each user....using f2 as we want to count number of hours where there was activity. If f2 is NAN then there was no activity
NUM_ACTIVE_DAYS_NEEDED = 5
entryCounts = motion.groupby(by='user').count()[['f2']]
#select users with at least 40 hours of non-stationary motion data
validmotion_Users= entryCounts[entryCounts['f2'] > 8 * NUM_ACTIVE_DAYS_NEEDED].reset_index()
print(f"Motion file has {validmotion_Users.shape[0]} users with at least {8 * NUM_ACTIVE_DAYS_NEEDED} hours of active motion data")


Motion file has 307 users with at least 40 hours of active motion data


#### 2.2 Left join valid user with full motion file...in effect this will remove non valid users from the motion dataframe


In [18]:
validmotion = pd.merge(validmotion_Users['user'],motion,on='user',how='left')

#### 2.3 Group motion file by user and by day....allowing us to compute summary features of motion per day for each user

In [28]:
validmotion['day'] = validmotion['datetime'].dt.date
dailyMotionAggs = validmotion.drop('datetime',axis=1).groupby(by=['user','day']).agg(['mean','min','max','std'])
# get rid of multi level column names and change to single flat column names which combines level0 and level1 naming
flatCols = dailyMotionAggs.columns.get_level_values(0) + '_' +  dailyMotionAggs.columns.get_level_values(1)
dailyMotionAggs.columns = flatCols


#### 2.4 After computing summary features for each individual day, combine all daily features into an overall feature to summarise motion for each user

In [29]:
summaryMotionAggs  = dailyMotionAggs.reset_index().groupby(by="user").mean()

In [25]:
pd.merge(motionFeatures.reset_index(),valid_questionnaire,on='user',how='left')

,user,f2_mean,f2_min,f2_max,f2_std,f3_mean,f3_min,f3_max,f3_std,f4_mean,...,Q34,Q35,Q36,Result1,Result2,Age,Gender,deviceID,registerDate,countryID
0,31,10.059483,9.712797,10.579330,0.243850,1.053516,0.532353,1.727772,0.386697,4.604868,...,100,100,100,94,96.0,6.0,2.0,7bf12be58456d518ff6c006fdbbae5e1,2016-01-17 19:30:50,IE
1,36,9.553528,9.252197,9.862114,0.337863,0.953485,0.512910,1.402428,0.457714,9.599101,...,0,50,0,48,24.0,2.0,1.0,f54d08fd081831552058f69a14e99a04,2016-01-18 15:36:34,NaN
2,50,9.914495,9.749846,10.292349,0.152828,0.786098,0.156163,1.898856,0.503003,6.221816,...,75,100,25,84,75.0,1.0,2.0,38439951a0c1fb700da3c2c106274b87,2016-01-19 09:09:57,GB
3,67,9.821368,9.695483,10.134630,0.142849,1.127611,0.740226,1.823095,0.352493,6.432974,...,75,50,75,87,78.0,3.0,2.0,3cbf9c038f0624d36dad148cccd5299e,2016-01-20 15:06:51,GB
4,69,9.842860,9.770496,9.935987,0.058666,0.626590,0.291216,1.051709,0.276763,5.576177,...,50,50,25,64,55.0,8.0,1.0,b385f4d26554b33b9be4bb6d871c4006,2016-01-20 17:56:03,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,5069,9.958318,9.815355,10.129122,0.115822,0.847515,0.422213,1.291379,0.329260,6.070524,...,50,50,50,50,50.0,7.0,2.0,189ab92af6f84ee2c20016cc12a8017d,2021-03-01 07:17:18,RO
303,5081,10.110907,9.813063,10.742833,0.299456,1.151792,0.335877,2.524432,0.712313,7.025376,...,75,-1,25,80,74.0,2.0,2.0,88b990fed4f52afa86e75918d3daec27,2021-03-03 05:00:20,AU
304,5180,9.992323,9.815692,10.301170,0.157999,0.772892,0.259688,1.419927,0.438452,4.929884,...,50,100,50,80,64.0,6.0,1.0,6639b2be484912e97caaac21301b839f,2021-05-03 22:19:51,US
305,5212,9.990184,9.755783,10.215525,0.135942,0.966458,0.436648,1.687259,0.383856,6.067085,...,50,25,0,35,41.0,6.0,1.0,9c104feafa261d01caf44a8b98634b2a,2021-05-31 03:45:33,AU
